inspired by https://github.com/jupyter-widgets/ipywidgets/blob/master/docs/source/examples/Lorenz%20Differential%20Equations.ipynb and others
# Exploring the Lorenz System of Differential Equations

In this Notebook we explore the Lorenz system of differential equations:

$$
\begin{aligned}
\dot{x} & = \sigma(y-x) \\
\dot{y} & = \rho x - y - xz \\
\dot{z} & = -\beta z + xy
\end{aligned}
$$

This is one of the classic systems in non-linear differential equations. It exhibits a range of different behaviors as the parameters (\\(\sigma\\), \\(\beta\\), \\(\rho\\)) are varied.

## Imports

First, we import the needed things from IPython, NumPy, Matplotlib and SciPy.

In [ ]:
%matplotlib inline

In [ ]:
from ipywidgets import interact, interactive, HBox, Layout,VBox
from IPython.display import clear_output, display, HTML

In [ ]:
import numpy as np
from scipy import integrate

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation

## Computing the trajectories and plotting the result

We define a function that can integrate the differential equations numerically and then plot the solutions. This function has arguments that control the parameters of the differential equation (\\(\sigma\\), \\(\beta\\), \\(\rho\\)), the numerical integration (`N`, `max_time`) and the visualization (`angle`).

In [ ]:
def solve_lorenz(numberOfTrajectories=10, anglex=0.0, angley=30.0, max_time=4.0, sigma=10.0, beta=8./3, rho=28.0):

    fig = plt.figure(figsize=(8, 6), dpi=80)
    ax = fig.add_axes([0, 0, 1, 1], projection='3d')
    ax.axis('off')

    # prepare the axes limits
    ax.set_xlim((-20, 20))
    ax.set_ylim((-30, 30))
    ax.set_zlim((5, 55))
    
    def lorenz_deriv(x_y_z, t0, sigma=sigma, beta=beta, rho=rho):
        """Compute the time-derivative of a Lorenz system."""
        x, y, z = x_y_z
        return [sigma * (y - x), x * (rho - z) - y, x * y - beta * z]

    # Choose random starting points, uniformly distributed from -15 to 15
    np.random.seed(1)
    x0 = -15 + 30 * np.random.random((numberOfTrajectories, 3))

    # Solve for the trajectories
    t = np.linspace(0, max_time, int(250*max_time))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0i, t)
                      for x0i in x0])
    
    # choose a different color for each trajectory
    colors = plt.cm.viridis(np.linspace(0, 1, numberOfTrajectories))

    for i in range(numberOfTrajectories):
        x, y, z = x_t[i,:,:].T
        lines = ax.plot(x, y, z, '-', c=colors[i])
        plt.setp(lines, linewidth=1)

    ax.view_init(angley, anglex)
    plt.show()

    return t, x_t

Let's call the function once to view the solutions. For this set of parameters, we see the trajectories swirling around two points, called attractors. 

In [ ]:
t, x_t = solve_lorenz(anglex=0, numberOfTrajectories=10)

Using IPython's `interactive` function, we can explore how the trajectories behave as we change the various parameters.

In [ ]:
w = interactive(solve_lorenz, anglex=(0.,360.), angley=(0.,360.), max_time=(0.1, 40.0), 
                numberOfTrajectories=(0,50), sigma=(0.0,50.0), rho=(0.0,50.0))
controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap'))
output = w.children[-1]
output.layout.height = '550px'
display(VBox([controls, output]))
#without this ping - the visualization would only update when someone actually interacts with the controls
w.children[1].value=w.children[1].value+1
w.children[1].value=w.children[1].value-1


The object returned by `interactive` is a `Widget` object and it has attributes that contain the current result and arguments:

In [ ]:
t, x_t = w.result

In [ ]:
w.kwargs

After interacting with the system, we can take the result and perform further computations. In this case, we compute the average positions in \\(x\\), \\(y\\) and \\(z\\).

In [ ]:
xyz_avg = x_t.mean(axis=1)

In [ ]:
xyz_avg.shape

Creating histograms of the average positions (across different trajectories) show that on average the trajectories swirl about the attractors.

In [ ]:
plt.hist(xyz_avg[:,0])
plt.title('Average $x(t)$');

In [ ]:
plt.hist(xyz_avg[:,1])
plt.title('Average $y(t)$');